In [9]:
from transpile_benchy.utilities.numerical_decomp import CircuitAnsatzDecomposer
from quantum_logical.conversiongain import ConversionGainThreeWaveTogether
import qutip as qt
import numpy as np

In [10]:
def transform_gate_to_qutrit(gate):
    """
    Transforms a 1Q qubit gate into a 1Q qutrit gate using the gf_isometry.

    Parameters:
    - gate: A 1Q qubit gate represented as a qutip Qobj.

    Returns:
    - A qutip Qobj representing the transformed 1Q qutrit gate.
    """
    # Define the isometry
    gf_isometry = qt.Qobj([[1, 0], [0, 0], [0, 1]])

    # Transform the gate using the isometry
    transformed_gate = gf_isometry * gate * gf_isometry.dag()

    return transformed_gate

See below, we have this messy optimization problem, where we are looking for something that will enable us to do $|g+f\rangle, |g-f\rangle$ parity checks. Hopefully, we can find something that looks like a $CZ_{gf}$ gate, but it turns out that trying to do 2 $iSWAP$ gates between g-e and e-f makes a SWAP.

In [11]:
from qiskit.circuit.library import UGate

H = ConversionGainThreeWaveTogether(
    gc=np.pi / 2,
    # gg=np.pi / 4,
    gg=0,
    phi_c1=-0.5 * np.pi,
    phi_c2=0 * np.pi,
    transmon_levels=3,
)
U3 = H.unitary(t=np.sqrt(2) / 2.0)

sqrt_iswap_gf = U3.full()
candidate_operator = np.eye(9)
u1 = transform_gate_to_qutrit(UGate(np.pi / 2, -np.pi / 2, 0).to_matrix())
u2 = transform_gate_to_qutrit(UGate(np.pi / 2, np.pi / 2, np.pi / 2).to_matrix())
u = np.kron(u1, u2)
candidate_operator = u @ candidate_operator
candidate_operator = sqrt_iswap_gf @ candidate_operator

u1 = transform_gate_to_qutrit(UGate(np.pi, -np.pi / 2, np.pi / 2).to_matrix())
u2 = transform_gate_to_qutrit(UGate(0, 0, np.pi).to_matrix())
u = np.kron(u1, u2)
candidate_operator = u @ candidate_operator
candidate_operator = sqrt_iswap_gf @ candidate_operator

u1 = transform_gate_to_qutrit(UGate(np.pi / 2, -np.pi / 2, -3 * np.pi / 2).to_matrix())
u2 = transform_gate_to_qutrit(UGate(np.pi / 2, 0, -np.pi / 2).to_matrix())
u = np.kron(u1, u2)
candidate_operator = u @ candidate_operator

U3_2Q = qt.Qobj(candidate_operator)
U3_2Q

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[-5.00000000e-01+6.65283447e-17j  0.00000000e+00+0.00000000e+00j
   5.00000000e-01-2.48949813e-17j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -1.69394048e-16+8.61273212e-17j  0.00000000e+00+0.00000000e+00j
   2.48949813e-17+8.32667268e-17j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j]
 [ 7.22495334e-17-5.00000000e-01j  0.00000000e+00+0.00000000e+00j
  -1.27760685e-16+5.00000000e-01j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   5.83717456e-17-3.06161700e-17j  0.00000000e+00+0.00000000e+00j
   5.55111512e-17+1.63672859e-16j]
 [ 0.0000000

Therefore, the idea is to put the above into my numerical decomposition tool. This might not work since there are too many parameters. But not sure what else to try and the moment.

The annoying part is that Qiskit does not allow Qutrits - so we have to make a custom class to handle the ansatz, whereas in the class proper it uses a qiskit.QuantumCircuit object.

In [12]:
class QutritAnsatz:
    def __init__(self, parameters=None, fixed_params=None):
        self.parameters = parameters or []
        self.fixed_params = fixed_params or {}
        self.ansatz = self._construct_ansatz()

    def _construct_ansatz(self):
        # Use the parameters to construct the ansatz
        # For the Hamiltonian, use self.parameters or self.fixed_params as needed
        # ... [rest of your code]
        U3_2Q = qt.Qobj(candidate_operator)
        return U3_2Q

    def assign_parameters(self, values):
        # Bind values to the parameters.
        # Ensure that fixed parameters are not modified.
        for p, v in zip(self.parameters, values):
            if p not in self.fixed_params:
                # Update the parameter with the new value
                pass  # Implement this

    def append(self, gate, qubits):
        # Implement this method to add a gate to the ansatz.
        pass

    def to_matrix(self):
        return self.ansatz.full()

    @property
    def parameters(self):
        return [p for p in self._parameters if p not in self.fixed_params]

    @parameters.setter
    def parameters(self, params):
        self._parameters = params